In [12]:
import pandas as pd
training_df = pd.read_csv('C:\\Users\Dell\Documents\Purdue\Coursework\Crossroad Challenge\DIWBB_Training.csv')

C:\Users\Dell\AppData\Local\Temp\ipykernel_7544\3422141235.py:2: DtypeWarning: Columns (13,14,15,16,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  training_df = pd.read_csv('C:\\Users\Dell\Documents\Purdue\Coursework\Crossroad Challenge\DIWBB_Training.csv')


In [13]:
training_df

,RecordID,ChampionshipYear,CustomerID,CustomerCity,CustomerState,CustomerZipCode,CustomerInstitutionAffinity,IsCustomerInNCAAMembership,HasCustomerClickedOrOpenedEmailsSixMonthsPrior,CustomerFirstWBBActionDate,...,EventSession,EventBeginDate,EventEndDate,HostingInstitution,FacilityName,FacilityDescription,FacilityCity,FacilityState,FacilityZipCode,ActivityType
0,4,2022,757758,Huntsville,TX,77340,(UNK),No,(N/A),10/28/13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Activity
1,14,2022,574752,Salem,OR,97304,(UNK),No,Yes,10/12/19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Activity
2,20,2022,741837,(UNK),UNK,(UNK),(UNK),No,Yes,10/28/13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Activity
3,21,2022,237618,(UNK),UNK,(UNK),(UNK),No,Yes,10/28/13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Activity
4,23,2022,799761,(UNK),UNK,(UNK),(UNK),No,(N/A),3/20/18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Activity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209261,1594423,2023,867155,Pine Bluff,AR,71603,(UNK),No,Yes,3/24/23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Activity
209262,1594427,2023,662341,Washington,WA,98155,(UNK),No,Yes,3/21/23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Activity
209263,1594428,2023,38418,(UNK),UNK,(UNK),(UNK),No,Yes,2/2/22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Activity
209264,1594445,2023,74611,(UNK),UNK,(UNK),(UNK),No,(N/A),10/28/13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Activity


In [7]:
nan_counts = training_set.isnull().sum()
print(nan_counts)

RecordID                                               0
ChampionshipYear                                       0
CustomerID                                             0
CustomerCity                                           0
CustomerState                                        598
CustomerZipCode                                        0
CustomerInstitutionAffinity                            0
IsCustomerInNCAAMembership                             0
HasCustomerClickedOrOpenedEmailsSixMonthsPrior         0
CustomerFirstWBBActionDate                         11065
CustomerFirstWBBPurchaseDate                       39301
CustomerLastWBBActionDate                          11065
CustomerLastWBBPurchaseDate                        39301
EventRoundName                                    191311
IsEventFinalSite                                  191311
EventSession                                      191311
EventBeginDate                                    191311
EventEndDate                   

In [14]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Standardize missing values
training_df.replace({'(UNK)': np.nan, '(N/A)': np.nan}, inplace=True)

# Drop columns that might not be useful for prediction
reduced_df = training_df.drop(['CustomerID', 'EventSession', 'EventBeginDate', 
                               'EventEndDate', 'HostingInstitution', 'FacilityName', 
                               'FacilityDescription', 'FacilityCity', 'FacilityState', 
                               'FacilityZipCode'], axis=1)

# Convert date columns to datetime and extract year for simplicity
reduced_df['CustomerFirstWBBActionDate'] = pd.to_datetime(reduced_df['CustomerFirstWBBActionDate'], errors='coerce').dt.year
reduced_df['CustomerLastWBBActionDate'] = pd.to_datetime(reduced_df['CustomerLastWBBActionDate'], errors='coerce').dt.year
reduced_df['CustomerFirstWBBPurchaseDate'] = pd.to_datetime(reduced_df['CustomerFirstWBBPurchaseDate'], errors='coerce').dt.year
reduced_df['CustomerLastWBBPurchaseDate'] = pd.to_datetime(reduced_df['CustomerLastWBBPurchaseDate'], errors='coerce').dt.year

# Fill missing values for numeric columns with the median
for col in ['CustomerFirstWBBActionDate', 'CustomerLastWBBActionDate', 
            'CustomerFirstWBBPurchaseDate', 'CustomerLastWBBPurchaseDate']:
    reduced_df[col].fillna(reduced_df[col].median(), inplace=True)

# Encode categorical variables
categorical_cols = reduced_df.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    reduced_df[col] = le.fit_transform(reduced_df[col].astype(str))
    label_encoders[col] = le

# Split the data into training and testing sets
X = reduced_df.drop('ActivityType', axis=1)
y = reduced_df['ActivityType']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a RandomForestClassifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
accuracy


0.986476800305825

In [16]:
# Load the test dataset
test_data_path = 'C:\\Users\Dell\Documents\Purdue\Coursework\Crossroad Challenge/DIWBB_Test.csv'
test_df = pd.read_csv(test_data_path)

# Standardize missing values in the test set
test_df.replace({'(UNK)': np.nan, '(N/A)': np.nan}, inplace=True)

# Drop columns that were not used in the model training
reduced_test_df = test_df.drop(['CustomerID', 'EventSession', 'EventBeginDate', 
                                'EventEndDate', 'HostingInstitution', 'FacilityName', 
                                'FacilityDescription', 'FacilityCity', 'FacilityState', 
                                'FacilityZipCode'], axis=1)

# Convert date columns to datetime and extract year
reduced_test_df['CustomerFirstWBBActionDate'] = pd.to_datetime(reduced_test_df['CustomerFirstWBBActionDate'], errors='coerce').dt.year
reduced_test_df['CustomerLastWBBActionDate'] = pd.to_datetime(reduced_test_df['CustomerLastWBBActionDate'], errors='coerce').dt.year
reduced_test_df['CustomerFirstWBBPurchaseDate'] = pd.to_datetime(reduced_test_df['CustomerFirstWBBPurchaseDate'], errors='coerce').dt.year
reduced_test_df['CustomerLastWBBPurchaseDate'] = pd.to_datetime(reduced_test_df['CustomerLastWBBPurchaseDate'], errors='coerce').dt.year

# Fill missing values for numeric columns with the median from the training set
for col in ['CustomerFirstWBBActionDate', 'CustomerLastWBBActionDate', 
            'CustomerFirstWBBPurchaseDate', 'CustomerLastWBBPurchaseDate']:
    reduced_test_df[col].fillna(reduced_df[col].median(), inplace=True)

# Encode categorical variables using the same encoders from the training set
for col, le in label_encoders.items():
    if col in reduced_test_df.columns:
        reduced_test_df[col] = reduced_test_df[col].map(lambda s: 'unknown' if s not in le.classes_ else s)
        le.classes_ = np.append(le.classes_, 'unknown')
        reduced_test_df[col] = le.transform(reduced_test_df[col].astype(str))


# Correct the code by removing references to 'ActivityType' for test data preprocessing
# Predict on the corrected test set
test_predictions = clf.predict(reduced_test_df)

# Add predictions to the test DataFrame
test_df['PredictedActivityType'] = test_predictions

# Display the first few rows with the predictions
test_df[['RecordID', 'CustomerID', 'PredictedActivityType']].head()



,RecordID,CustomerID,PredictedActivityType
0,26355,552730,1
1,26641,84338,1
2,26836,656065,1
3,26900,419848,1
4,26948,338825,1


In [17]:
# Reverse the encoding to get the original category names for 'PredictedActivityType'
predicted_activity_type_names = label_encoders['ActivityType'].inverse_transform(test_df['PredictedActivityType'])

# Update the 'PredictedActivityType' column with the original category names
test_df['PredictedActivityType'] = predicted_activity_type_names

# Display the first few rows with the predictions
test_df[['RecordID', 'CustomerID', 'PredictedActivityType']].head()


,RecordID,CustomerID,PredictedActivityType
0,26355,552730,No Activity
1,26641,84338,No Activity
2,26836,656065,No Activity
3,26900,419848,No Activity
4,26948,338825,No Activity


In [22]:
# Load the submission template to understand its structure
template_path = 'C:\\Users\Dell\Documents\Purdue\Coursework\Crossroad Challenge/DIWBB_Submission_Template.csv'
submission_template = pd.read_csv(template_path)


# Prepare the submission DataFrame
submission_df = test_df[['RecordID', 'PredictedActivityType']].copy()
submission_df.rename(columns={'PredictedActivityType': 'ActivityType'}, inplace=True)

# Ensure the order of RecordID matches the template, if necessary
# This step is important if the order of RecordIDs in the submission must match the template exactly
submission_df = submission_df.set_index('RecordID').reindex(submission_template['RecordID']).reset_index()

# Save the submission DataFrame as a CSV file
submission_csv_path = 'C:\\Users\Dell\Documents\Purdue\Coursework\Crossroad Challenge/DIWBB_Submission.csv'
submission_df.to_csv(submission_csv_path, index=False)